In [ ]:
import time
import json
import ast
import os
import datetime
from rich.progress import track

import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
# import seaborn as sns

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz
from rich.progress import track
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
%matplotlib inline
pd.set_option('display.max_rows', 500)
import json
import cv2
import os
import multiprocessing as mp    

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dl.utils.io_utils import normalize_image
import imageio
from pathlib import Path
from dl.utils.mask_utils import fill_nan_geometries

%matplotlib inline
from cv.core.image_quality_server_side import ImageQuality

# from cv.core.image_quality_server_side_halo import ImageQuality
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

In [ ]:
directory = '/data2/jupiter/datasets/halo_rgb_stereo_train_v6_1/'
csv_name = 'master_annotations.csv' 
from dl.dataset.datamodes.npz.rgbd import RGBDNPZ
dataloader = RGBDNPZ(directory)

In [ ]:
df = pd.read_csv(directory + csv_name)
df = df.drop_duplicates('unique_id')
df = df.sample(frac=1, ignore_index=True)
fill_nan_geometries(df)

In [ ]:
df_1 = df[(df['camera_geometry_label'] == 'puk30') & (df['camera_location'] == 'T06') &  (df['camera_location_right'] == 'T08')].head(10)
df_small = df.groupby(['camera_geometry_label', 'camera_location','camera_location_right']).head(10)

In [ ]:
total_count = defaultdict(lambda:0)
ego_count = {}
for i, row in track(df_1.iterrows(), total=len(df_1)):
    if row["camera_location"][0] == "I":
        continue  # No implement camera masks for now
    
    ego_ids = []
    for k, v in json.loads(row["label_map"]).items():
        if v[:3] == "Ego":
            ego_ids.append(int(k))
    l_cam = row['camera_location']
    r_cam = row['camera_location_right']
    label = dataloader.get_artifacts(row)['label']
    ego_key = (row["camera_geometry_label"], l_cam, r_cam)
    total_count[ego_key] += 1
    for id in ego_ids:
        label_is_ego = (label == id).astype(np.int32)
        if ego_key not in ego_count:
            ego_count[ego_key] = label_is_ego
        else:
            ego_count[ego_key] += label_is_ego
for ego_key in ego_count.keys():
    ego_count[ego_key] = ego_count[ego_key].astype(np.float32) / total_count[ego_key]


In [ ]:
res = dataloader.get_artifacts(df_1.iloc[5])
plt.imshow(normalize_image(res['image'], hdr_mode=True))
plt.imshow(res['label'])

In [ ]:
from scipy.spatial import ConvexHull
from skimage.draw import polygon
from skimage.morphology import binary_dilation

def expand_mask(image: np.ndarray) -> np.ndarray:
    image_binary = (image > 0.1).astype(np.uint8)
    x, y = np.where(image_binary > 0)
    points = np.stack([x, y], axis=1)
    if len(points) == 0:
        return np.zeros_like(image, dtype=np.uint8)
    else:
        hull = points[ConvexHull(points).vertices]
        rr, cc = polygon(hull[:, 0], hull[:, 1], image_binary.shape)
        image_binary[rr, cc] = 1
        image_binary_dilate = binary_dilation(image_binary, footprint=np.ones((51, 51)))
        image_out = (image_binary_dilate > 0).astype(np.uint8) * 255
        return image_out
out = expand_mask(ego_count[('puk30', 'T06', 'T08')])
plt.imshow(out)

In [ ]:
def plot_several_images(img_list, titles):
    cols = 4
    rows = (len(img_list) + 3) // cols
    fig, ax = plt.subplots(rows, cols, squeeze=False, figsize = (16, rows * 4))
    for i, img in enumerate(img_list):
        ax[i // cols][i % cols].imshow(img)
        ax[i // cols][i % cols].set_title(titles[i])
plot_several_images(ego_count.values(), list(ego_count.keys()))

In [ ]:
image = list(ego_count.values())[0]

In [ ]:
image > .5

In [ ]:
len(contours)

In [ ]:
cv2.convexHull(contours[0])

In [ ]:
image_binary = np.uint8(image > .5) * 255
plt.imshow(image_binary)

In [ ]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [ ]:
x, y, _ = np.where(image_binary > 0)

In [ ]:
points = np.stack([x, y], axis=1)

In [ ]:
hull = ConvexHull(points)

In [ ]:
hull # TODO

In [ ]:
plot_several_images([image], ['a'])